# Méthodes par remplacement pour la data augmentation

## Install & Packages

In [1]:
!pip install Sentencepiece
!pip install transformers
!pip install nltk
!python -m spacy download fr_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-25 07:50:01.401740: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 07:50:02.722461: W tensorflow/compiler/tf2tensorrt/utils/py_uti

In [2]:
from transformers import CamembertModel, CamembertTokenizer
import nltk
from nltk.corpus import wordnet as wn
import random
import spacy
from transformers import pipeline
import os

## Remplacement par synonymes

In [3]:
nltk.download('omw-1.4')
nltk.download('wordnet')

"""
  Cette fonction prend un mot en entrée et renvoie une liste de synonymes pour ce mot.
  
  La fonction utilise le module WordNet de la bibliothèque NLTK pour récupérer les synsets (ensemble de synonymes) associés au mot donné en entrée.
  Pour chaque synset, la fonction ajoute les noms des lemmes (formes canoniques) en français à une liste de synonymes, où n est la valeur de l'argument k.

  Input:
    word : str - Le mot pour lequel les synonymes doivent être trouvés
    k : int - (facultatif) Le nombre de synonymes à renvoyer (4 par défaut)

  Output:
    synonyms : list - La liste des synonymes pour le mot donné en entrée
"""
def liste_synonymes(word, k=4):
    synonyms = set()
    for synset in wn.synsets(word, lang='fra'):
        for lemma in synset.lemmas(lang='fra'):
            synonyms.add(lemma.name())
            if len(synonyms) == k:
                break
        if len(synonyms) == k:
            break
    return list(synonyms)


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
# Exemple d'utilisation
word = "rapide"
synonymes = liste_synonymes(word)
print(f"Les synonymes de '{word}' sont : {', '.join(synonymes)}")

Les synonymes de 'rapide' sont : vif, rapide, agile, actif


## Remplacement par \<mask\>

In [6]:
nlp = spacy.load('fr_core_news_sm')

"""
  Cette fonction prend une chaîne de caractères en entrée et retourne une liste de phrases où un mot aléatoire a été masqué par la balise <mask>.
  Les mots masqués sont choisis parmi les mots alphabétiques (en excluant les noms propres et les ponctuations) de chaque phrase.

  Input:
    text : str - Le texte en entrée à masquer

  Output:
    masked_sentences : list - La liste des phrases où un mot a été masqué
"""
def mask_sentences(text):
    doc = nlp(text)
    masked_sentences = []

    for sent in doc.sents:
        tokens = [token for token in sent]
        # Filtrer les tokens pour conserver uniquement les mots (en excluant les noms propres et les ponctuations)
        words = [token for token in tokens if token.is_alpha and token.pos_ != 'PROPN']

        if words:
            # Choisir un mot au hasard parmi les mots candidats
            word_to_mask = random.choice(words)
            masked_tokens = ['<mask>' + token.whitespace_ if token == word_to_mask else token.text_with_ws for token in tokens]
            masked_sentence = "".join(masked_tokens)
        else:
            # Si aucune liste de mots candidats n'est trouvée, conserver la phrase telle quelle
            masked_sentence = sent.text

        masked_sentences.append(masked_sentence)

    return masked_sentences

"""
  Cette fonction prend une liste de phrases masquées en entrée, remplace chaque balise <mask> par un mot prédit par le modèle CamemBERT et retourne une nouvelle liste de phrases.

  Input:
    masked_sentences_list : list - La liste des phrases masquées à remplir avec des prédictions de mots

  Output:
    filled_sentences_list : list - La liste des phrases complètes où les balises <mask> ont été remplacées par des mots
"""
def fill_mask(masked_sentences_list):
    # Charger le modèle CamemBERT
    camembert_fill_mask = pipeline('fill-mask', model='camembert-base', tokenizer='camembert-base')

    filled_sentences_list = []

    for masked_sentence in masked_sentences_list:
        if '<mask>' in masked_sentence:
            result = camembert_fill_mask(masked_sentence)
            filled_sentence = result[0]['sequence']
            filled_sentences_list.append(filled_sentence)
        else:
            filled_sentences_list.append(masked_sentence)

    return filled_sentences_list

"""
  Cette fonction prend un chemin de fichier en entrée, lit le contenu du fichier, applique les fonctions mask_sentences et fill_mask sur le texte, 
  puis écrit le résultat dans un nouveau fichier. La fonction utilise le nom du fichier d'entrée pour générer le nom du fichier de sortie en 
  ajoutant le suffixe "_masked". 

  Input:
    file_path : str - Le chemin d'accès complet du fichier d'entrée à transformer

  Output:
    str - Une chaîne de caractères qui indique que le fichier de sortie a été créé avec succès
""" 
def transform_text(file_path):
    with open("output.txt", "r") as fichier:
        text = fichier.read()
        
    sentences = mask_sentences(text)
    filled_sentences = fill_mask(sentences)
    final_text = "\n".join(filled_sentences)

    # Déterminer le nom du fichier de sortie
    file_name, file_ext = os.path.splitext(file_path)
    file_ext = f"_masked{file_ext}"
    file_out = file_name + file_ext

    with open(file_out, "w") as output_file:
        output_file.write(final_text)

    return "Le fichier " + file_out + " a bien été crée."


In [7]:
transform_text("output.txt")

'Le fichier output_masked.txt a bien été crée.'